<a href="https://colab.research.google.com/github/caseyeaston/BEA_PipelineEngagementAnalysis/blob/main/BEA_PipelineEngagementCleaningFinalv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import libraries
import pandas as pd
import numpy as np

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

In [3]:
# Define file paths
base_path = '/content/drive/MyDrive/Work/BEA/2025 BEA Data Project Shared Folder/Data/(Main) Data Sources/Existing/PPBEA Pipeline/CSVs/'
file_paths = {
    '2019-2020': f'{base_path}2019-2020_PPBEA Pipeline_Engagement.csv',
    '2020-2021': f'{base_path}2020-2021_PPBEA Pipeline_Engagement.csv',
    '2021-2022': f'{base_path}2021-2022_PPBEA Pipeline_Engagement.csv',
    '2022-2023': f'{base_path}2022-2023_PPBEA Pipeline_Engagement.csv',
    '2023-2024': f'{base_path}2023-2024_PPBEA Pipeline_Engagement.csv',
    '2024-2025': f'{base_path}2024-2025_PPBEA Pipeline_Engagement.csv',
}

# Load all CSV files
dfs = {}
for year, path in file_paths.items():
    df = pd.read_csv(path)
    df = df.dropna(how='all')  # Remove completely empty rows
    df['School Year'] = year  # Add school year identifier
    dfs[year] = df

# Combine all dataframes
dfmain = pd.concat(dfs.values(), ignore_index=True)

In [4]:
# Rename PPBEA Member to District
dfmain = dfmain.rename(columns={'PPBEA Member': 'District'})

In [5]:
# Drop district columns and other unwanted columns
columns_to_drop = [
    ' ',  # Unnamed column
    'Calhan District RJ-1', 'Harrison District 2', 'Widefield District 3',
    'Fountain Ft.Carson District 8', 'Colorado Springs District 11',
    'Cheyenne Mountain District 12', 'Manitou Springs District 14',
    'Academy District 20', 'Ellicott District 22', 'Peyton District 23JT',
    'Lewis Palmer District 38', 'El Paso County District 49',
    'Colorado Springs Early College (CSEC)', 'CO Digital BOCES PPOS & CPA',
    'Eastlake High School', 'Banning Lewis Ranch', 'Atlas Prep',
    'Woodland Park School District',
    'Unnamed: 24',
    'Career Rep Email', 'Follow-up Task: ', 'Employer post Internship',
    'Sponsor Email', 'Placed into Employment Post Internship',
    'Staff Interactions with Businesses', 'Career Rep First Name',
    'Career Rep Last Name', 'Opp Number', 'Task Number',
    'PPBEA Staff Assigned', 'Next Action', 'PPBEA Notes',
    'Notes: Student Name, Duration, School Name, Sponsor Name, Teacher Name, Flags'
]
dfmain = dfmain.drop(columns=columns_to_drop)

In [6]:
# Merge duplicate columns
dfmain['Pro101 Certificates Earned'] = dfmain['Pro101 Certificates Earned'].fillna(
    dfmain['Professionalism 101 Certificates Earned']
)
dfmain = dfmain.drop(columns=[
    'Professionalism 101 Certificates Earned',
])

In [7]:
# Convert numeric columns
numeric_cols = [
    'Complete Student Trainings',
    'Complete Staff Trainings',
    'Complete Student Interactions',
    'Complete Student Internships',
    'Internships in Progress',
    'Pending Student Interactions',
    'Declined or Cancelled Student Interactions',
    'Pro101 Certificates Earned'
]

for col in numeric_cols:
    dfmain[col] = pd.to_numeric(dfmain[col], errors='coerce')

# Fill nulls with 0 for numeric columns
for col in numeric_cols:
    dfmain[col] = dfmain[col].fillna(0)

In [8]:
# Remove summary and empty rows
dfmain = dfmain[
    ((dfmain['Complete Student Interactions'] <= 2000) | (dfmain['Complete Student Interactions'].isna())) &
    (dfmain['Event Title'].notna())
]

In [9]:
# Convert date columns to datetime
dfmain['Initiation Date'] = pd.to_datetime(dfmain['Initiation Date'], errors='coerce')
dfmain['Status Update Date'] = pd.to_datetime(dfmain['Status Update Date'], errors='coerce')
dfmain['Event Date or Start Date'] = pd.to_datetime(dfmain['Event Date or Start Date'], errors='coerce')

In [10]:
# Standardize WBL Opportunity Type
dfmain['WBL Opportunity Type'] = dfmain['WBL Opportunity Type'].replace({
    "Speaker's Bureau": "Speakers Bureau"
})

# Drop one-off WBL type
dfmain = dfmain[dfmain['WBL Opportunity Type'] != 'Jobs/Training/Apprenticeship']

# Clean text columns (strip whitespace/newlines)
dfmain = dfmain.copy()
dfmain['Business Champion Name'] = dfmain['Business Champion Name'].str.strip()
dfmain = dfmain.rename(columns={'Student and Sponsor\nor School POC Name': 'Student Sponsor Name'})
dfmain['Student Sponsor Name'] = dfmain['Student Sponsor Name'].str.strip()

In [11]:
# Checking shape
dfmain.shape

(10068, 19)

# Date Errors and Nulls

In [12]:
# Fix unrealistic dates (before 2018)
date_cols = ['Initiation Date', 'Status Update Date', 'Event Date or Start Date']
cutoff_date = pd.Timestamp('2018-01-01')

for col in date_cols:
    unrealistic = dfmain[col] < cutoff_date
    count = unrealistic.sum()
    if count > 0:
        print(f"Setting {count} unrealistic dates to null in {col}")
        dfmain.loc[unrealistic, col] = pd.NaT

# Fill remaining nulls in date columns using cascade logic
dfmain['Initiation Date'] = dfmain['Initiation Date'].fillna(dfmain['Event Date or Start Date']).fillna(dfmain['Status Update Date'])
dfmain['Status Update Date'] = dfmain['Status Update Date'].fillna(dfmain['Event Date or Start Date']).fillna(dfmain['Initiation Date'])

# Create Derived Event Date column with fallback logic
# Priority: Event Date or Start Date → Status Update Date → Initiation Date
dfmain['Derived Event Date'] = dfmain['Event Date or Start Date'].fillna(
    dfmain['Status Update Date']
).fillna(
    dfmain['Initiation Date']
)

Setting 4 unrealistic dates to null in Initiation Date
Setting 7 unrealistic dates to null in Status Update Date


# 'Placement Status' Parent Column & Separating Declined/Cancelled

In [13]:
# Create parent category column
def categorize_placement_status(status):
    if status == 'Completed':
        return 'Completed'
    elif status in ['Cancelled-COVID', 'Cancelled-Weather', 'Cancelled-Illness']:
        return 'Cancelled'
    elif status in ['Initial Contact Made', 'Pending-Scheduling', 'Scheduled Interview',
    'Internship In Process', 'Scheduled Event (Pending Completion)']:
        return 'Pending'
    else:
        return 'Declined'

dfmain['Placement Status Category'] = dfmain['Placement Status'].apply(categorize_placement_status)

# Verify
dfmain['Placement Status Category'].value_counts()

,count
Placement Status Category,
Completed,7549
Declined,2161
Pending,277
Cancelled,81


In [14]:
# Create Cancelled column and split Declined/Cancelled
dfmain['Cancelled Student Interactions'] = 0
dfmain = dfmain.rename(columns={
    'Declined or Cancelled Student Interactions': 'Declined Student Interactions'
})

# Move Cancelled values to correct column based on Placement Status Category
cancelled_mask = dfmain['Placement Status Category'] == 'Cancelled'
dfmain.loc[cancelled_mask, 'Cancelled Student Interactions'] = dfmain.loc[cancelled_mask, 'Declined Student Interactions']
dfmain.loc[cancelled_mask, 'Declined Student Interactions'] = 0

# Convert 'Cancelled Student Interactions' to float datatype
dfmain['Cancelled Student Interactions'] = dfmain['Cancelled Student Interactions'].astype(float)

# Redefine numeric columns
numeric_cols = [
    'Complete Student Trainings',
    'Complete Staff Trainings',
    'Complete Student Interactions',
    'Complete Student Internships',
    'Internships in Progress',
    'Pending Student Interactions',
    'Declined Student Interactions',
    'Cancelled Student Interactions',
    'Pro101 Certificates Earned'
]

# Pro101

In [15]:
# from rapidfuzz import fuzz

# # Extract student name (before "/") from Student Sponsor Name
# def extract_student_name(name):
#     if pd.isna(name):
#         return ''
#     name_str = str(name)
#     if '/' in name_str:
#         return name_str.split('/')[0].strip()
#     else:
#         return name_str.strip()

# # Add temporary column for student names
# dfmain['Student Name'] = dfmain['Student Sponsor Name'].apply(extract_student_name)

# # Find rows with Pro101 cert earned during other events
# pro101_during_other_event = dfmain[
#     (dfmain['Pro101 Certificates Earned'] > 0) &
#     (dfmain['WBL Opportunity Type'] != 'Professionalism 101 Training')
# ]

# print(f"Rows with Pro101 cert earned during OTHER events: {len(pro101_during_other_event)}")

# # Check if any already have matching standalone Pro101 rows
# potential_duplicates = []

# for idx, row in pro101_during_other_event.iterrows():
#     # Get student name from this row
#     student_name = row['Student Name']

#     if not student_name:  # Skip if no student name
#         continue

#     # Look for Pro101 Training rows with fuzzy match on student name
#     pro101_rows = dfmain[dfmain['WBL Opportunity Type'] == 'Professionalism 101 Training']

#     for pro101_idx, pro101_row in pro101_rows.iterrows():
#         pro101_student_name = pro101_row['Student Name']

#         if not pro101_student_name:
#             continue

#         # Fuzzy match on student names
#         similarity = fuzz.ratio(student_name.lower(), pro101_student_name.lower())

#         if similarity >= 85:  # 85% threshold
#             potential_duplicates.append((idx, pro101_idx, similarity))
#             break  # Found a match, move to next row

# print(f"\nRows that already have standalone Pro101 records: {len(potential_duplicates)}")

# if len(potential_duplicates) > 0:
#     print("\nSample matches (first 10):")
#     for orig_idx, pro101_idx, similarity in potential_duplicates[:10]:
#         print(f"\nOriginal event row {orig_idx} matches Pro101 row {pro101_idx} (similarity: {similarity}%)")
#         print(f"  Original: {dfmain.loc[orig_idx, 'Student Name']} - {dfmain.loc[orig_idx, 'WBL Opportunity Type']}")
#         print(f"  Pro101:   {dfmain.loc[pro101_idx, 'Student Name']} - {dfmain.loc[pro101_idx, 'WBL Opportunity Type']}")

In [16]:
# Drop the 2 existing Pro101 rows that we'll recreate from their matching events (found using fuzzy matching above)
dfmain = dfmain.drop([1098, 540])

# Find rows that need to be split (Pro101 earned during another event)
rows_to_split = dfmain[
    (dfmain['Pro101 Certificates Earned'] > 0) &
    (dfmain['WBL Opportunity Type'] != 'Professionalism 101 Training')
].copy()

# Create new Pro101 rows
new_pro101_rows = []

for idx, row in rows_to_split.iterrows():
    pro101_row = row.copy()
    pro101_row['Placement Status'] = 'Completed'
    pro101_row['Placement Status Category'] = 'Completed'
    pro101_row['Business Champion Name'] = 'PPBEA'
    pro101_row['Event Title'] = 'PPBEA Professionalism 101 Course'
    pro101_row['WBL Opportunity Type'] = 'Professionalism 101 Training'
    pro101_row['Complete Student Interactions'] = 1
    pro101_row['Complete Student Trainings'] = 0
    pro101_row['Complete Staff Trainings'] = 0
    pro101_row['Complete Student Internships'] = 0
    pro101_row['Internships in Progress'] = 0
    pro101_row['Pending Student Interactions'] = 0
    pro101_row['Declined Student Interactions'] = 0
    pro101_row['Cancelled Student Interactions'] = 0
    pro101_row['Pro101 Certificates Earned'] = 0
    new_pro101_rows.append(pro101_row)

# Add new Pro101 rows to dfmain
dfmain = pd.concat([dfmain, pd.DataFrame(new_pro101_rows)], ignore_index=True)

# Transfer 'Pro101 Certificated Earned' to 'Complete Student Interactions'
pro101_completed_wrong = dfmain[
    (dfmain['Placement Status'] == 'Completed') &
    (dfmain['WBL Opportunity Type'] == 'Professionalism 101 Training') &
    (dfmain['Complete Student Interactions'] == 0)
]

dfmain.loc[pro101_completed_wrong.index, 'Complete Student Interactions'] = 1

# Drop Pro101 column (no longer needed)
dfmain = dfmain.drop(columns=['Pro101 Certificates Earned'])

print(f"Created {len(new_pro101_rows)} new Pro101 rows")
dfmain.shape

Created 61 new Pro101 rows


(10127, 21)

# Placement Status & Numeric Columns Mismatch

### Main

In [17]:
# Redefine numeric columns
numeric_cols = [
    'Complete Student Trainings',
    'Complete Staff Trainings',
    'Complete Student Interactions',
    'Complete Student Internships',
    'Internships in Progress',
    'Pending Student Interactions',
    'Declined Student Interactions',
    'Cancelled Student Interactions',
]

# WBL to column mapping for Completed rows
wbl_to_column_map_completed = {
    'Staff Training': 'Complete Staff Trainings',
    'Regional Advisory Meeting': 'Complete Staff Trainings',
    'Site Visit - Staff': 'Complete Staff Trainings',
    'Student Training': 'Complete Student Trainings',
    'Professionalism 101 Training': 'Complete Student Interactions',
    'Informational Interview Video': 'Complete Student Interactions',
    'Career Story Video': 'Complete Student Interactions',
    'e-WBL Informational Interview': 'Complete Student Interactions',
    'e-WBL Class Presentation': 'Complete Student Interactions',
    'Job Fair': 'Complete Student Interactions',
    'Class/Group Mentorship': 'Complete Student Interactions',
    'Industry Sponsored Project': 'Complete Student Interactions',
    'Class Presentation': 'Complete Student Interactions',
    'Job Shadow': 'Complete Student Interactions',
    'Site Visit': 'Complete Student Interactions',
    'Speakers Bureau': 'Complete Student Interactions',
    'Event': 'Complete Student Interactions',
    'Individual Mentorship': 'Complete Student Interactions',
    'Paid Job': 'Complete Student Interactions',
    'Internship 60': 'Complete Student Internships',
    'Internship 120': 'Complete Student Internships',
    'Internship 320': 'Complete Student Internships',
    'Apprenticeship': 'Complete Student Interactions'
}

# WBL to column mapping for Pending rows
wbl_to_column_map_pending = {
    'Professionalism 101 Training': 'Pending Student Interactions',
    'Career Story Video': 'Pending Student Interactions',
    'e-WBL Informational Interview': 'Pending Student Interactions',
    'Industry Sponsored Project': 'Pending Student Interactions',
    'Class Presentation': 'Pending Student Interactions',
    'Job Shadow': 'Pending Student Interactions',
    'Site Visit': 'Pending Student Interactions',
    'Speakers Bureau': 'Pending Student Interactions',
    'Event': 'Pending Student Interactions',
    'Internship 60': 'Internships in Progress',
    'Internship 120': 'Internships in Progress',
    'Internship 320': 'Internships in Progress',
    'Apprenticeship': 'Pending Student Interactions'
}

### Fixing Double-Counting

In [18]:
# Fix double-counting: rows with values in multiple numeric columns
dfmain['num_cols_with_values'] = (dfmain[numeric_cols] > 0).sum(axis=1)
rows_with_multiple = dfmain['num_cols_with_values'] > 1

# Zero out Complete Student Interactions for rows with Trainings + Interactions
dfmain.loc[rows_with_multiple, 'Complete Student Interactions'] = 0

# Drop helper column
dfmain = dfmain.drop(columns=['num_cols_with_values'])

# Create a column for numeric sum
dfmain['_numeric_sum'] = dfmain[numeric_cols].sum(axis=1)

### Verify 'Placement Status' Matches Numeric Column Values

In [19]:
# Investigate Placement Status & Numeric Column Mismatches

# Define column groups by placement status category
complete_cols = ['Complete Student Trainings', 'Complete Staff Trainings',
                 'Complete Student Interactions', 'Complete Student Internships']
declined_cols = ['Declined Student Interactions']
cancelled_cols = ['Cancelled Student Interactions']
pending_cols = ['Pending Student Interactions', 'Internships in Progress']

# All numeric columns
all_numeric_cols = complete_cols + declined_cols + cancelled_cols + pending_cols

# Find mismatches
mismatches = []

for idx in dfmain.index:
    row = dfmain.loc[idx]

    # Skip all-zero rows (handled separately in fill-all-zeros section)
    if (row[all_numeric_cols] == 0).all():
        continue

    category = row['Placement Status Category']

    # Check what columns have values
    has_complete = (row[complete_cols] > 0).any()
    has_declined = row['Declined Student Interactions'] > 0
    has_cancelled = row['Cancelled Student Interactions'] > 0
    has_pending = (row[pending_cols] > 0).any()

    # Determine if there's a mismatch
    is_mismatch = False
    mismatch_type = None

    if category == 'Completed':
        if has_declined or has_cancelled or has_pending:
            is_mismatch = True
            mismatch_type = 'Completed status but has Declined/Cancelled/Pending values'
        elif not has_complete:
            is_mismatch = True
            mismatch_type = 'Completed status but no Complete values'

    elif category == 'Declined':
        if has_complete or has_cancelled or has_pending:
            is_mismatch = True
            mismatch_type = 'Declined status but has Complete/Cancelled/Pending values'
        elif not has_declined:
            is_mismatch = True
            mismatch_type = 'Declined status but no Declined values'

    elif category == 'Cancelled':
        if has_complete or has_declined or has_pending:
            is_mismatch = True
            mismatch_type = 'Cancelled status but has Complete/Declined/Pending values'
        elif not has_cancelled:
            is_mismatch = True
            mismatch_type = 'Cancelled status but no Cancelled values'

    elif category == 'Pending':
        if has_complete or has_declined or has_cancelled:
            is_mismatch = True
            mismatch_type = 'Pending status but has Complete/Declined/Cancelled values'
        elif not has_pending:
            is_mismatch = True
            mismatch_type = 'Pending status but no Pending values'

    if is_mismatch:
        mismatches.append({
            'index': idx,
            'Placement Status': row['Placement Status'],
            'Placement Status Category': category,
            'Mismatch Type': mismatch_type,
            'Complete Student Trainings': row['Complete Student Trainings'],
            'Complete Staff Trainings': row['Complete Staff Trainings'],
            'Complete Student Interactions': row['Complete Student Interactions'],
            'Complete Student Internships': row['Complete Student Internships'],
            'Declined Student Interactions': row['Declined Student Interactions'],
            'Cancelled Student Interactions': row['Cancelled Student Interactions'],
            'Pending Student Interactions': row['Pending Student Interactions'],
            'Internships in Progress': row['Internships in Progress'],
            'WBL Opportunity Type': row['WBL Opportunity Type'],
            'School Year': row['School Year']
        })

# Create dataframe
if len(mismatches) > 0:
    mismatches_df = pd.DataFrame(mismatches)

    print(f"Total mismatches found (excluding all-zero rows): {len(mismatches_df)}")
    print("\nBreakdown by Mismatch Type:")
    print(mismatches_df['Mismatch Type'].value_counts())

    print("\nBreakdown by Placement Status Category:")
    print(mismatches_df['Placement Status Category'].value_counts())

    print("\nBreakdown by School Year:")
    print(mismatches_df['School Year'].value_counts())

    # print("\nFirst 20 mismatches:")
    # display(mismatches_df.head(20))

Total mismatches found (excluding all-zero rows): 30

Breakdown by Mismatch Type:
Mismatch Type
Pending status but has Complete/Declined/Cancelled values     28
Completed status but has Declined/Cancelled/Pending values     2
Name: count, dtype: int64

Breakdown by Placement Status Category:
Placement Status Category
Pending      28
Completed     2
Name: count, dtype: int64

Breakdown by School Year:
School Year
2019-2020    28
2021-2022     1
2023-2024     1
Name: count, dtype: int64


In [20]:
# Fix Placement Status & Numeric Column Mismatches

# Track fixes
fixed_rows = []

# Fix the mismatches
for idx in mismatches_df['index']:
    row = dfmain.loc[idx]
    category = row['Placement Status Category']
    wbl_type = row['WBL Opportunity Type']

    # Check what columns have values
    has_complete = (row[complete_cols] > 0).any()
    has_declined = row['Declined Student Interactions'] > 0
    has_cancelled = row['Cancelled Student Interactions'] > 0
    has_pending = (row[pending_cols] > 0).any()

    # Case 1: Pending status but has Complete/Declined/Cancelled values
    if category == 'Pending' and (has_complete or has_declined or has_cancelled):
        # Change to Completed status
        dfmain.loc[idx, 'Placement Status'] = 'Completed'
        dfmain.loc[idx, 'Placement Status Category'] = 'Completed'

        fixed_rows.append({
            'index': idx,
            'Fix Type': 'Changed Pending → Completed',
            'WBL Opportunity Type': wbl_type,
            'Old Status': 'Pending',
            'New Status': 'Completed'
        })

    # Case 2: Completed status but has Declined/Cancelled values
    elif category == 'Completed' and (has_declined or has_cancelled):
        # Move values to correct Complete column based on WBL mapping
        if wbl_type in wbl_to_column_map_completed:
            correct_column = wbl_to_column_map_completed[wbl_type]

            # Move Declined value if present
            if has_declined:
                value = row['Declined Student Interactions']
                dfmain.loc[idx, correct_column] = dfmain.loc[idx, correct_column] + value
                dfmain.loc[idx, 'Declined Student Interactions'] = 0

                fixed_rows.append({
                    'index': idx,
                    'Fix Type': 'Moved Declined → Complete',
                    'WBL Opportunity Type': wbl_type,
                    'Value Moved': value,
                    'To Column': correct_column
                })

            # Move Cancelled value if present
            if has_cancelled:
                value = row['Cancelled Student Interactions']
                dfmain.loc[idx, correct_column] = dfmain.loc[idx, correct_column] + value
                dfmain.loc[idx, 'Cancelled Student Interactions'] = 0

                fixed_rows.append({
                    'index': idx,
                    'Fix Type': 'Moved Cancelled → Complete',
                    'WBL Opportunity Type': wbl_type,
                    'Value Moved': value,
                    'To Column': correct_column
                })

# # Create summary dataframe
# if len(fixed_rows) > 0:
#     fixed_df = pd.DataFrame(fixed_rows)
#     print(f"Fixed {len(fixed_rows)} placement status mismatches")
#     print("\nBreakdown by Fix Type:")
#     print(fixed_df['Fix Type'].value_counts())
#     print("\nFixed rows:")
#     display(fixed_df)
# else:
#     print("No fixes needed")

### Verifying Correct Mapping from WBL Type to Respective Numeric Column

In [21]:
# # Define the complete columns
# complete_cols = ['Complete Student Trainings', 'Complete Student Interactions',
#                  'Complete Student Internships']

# # Analyze Completed rows
# completed_rows = dfmain[dfmain['Placement Status Category'] == 'Completed'].copy()

# print("="*100)
# print("COMPLETED ROWS - WBL Type to Numeric Column Distribution")
# print("="*100)

# for wbl_type in sorted(completed_rows['WBL Opportunity Type'].dropna().unique()):
#     wbl_subset = completed_rows[completed_rows['WBL Opportunity Type'] == wbl_type]

#     # Only show if there are non-zero values
#     has_values = (wbl_subset[complete_cols] > 0).any().any()

#     if has_values:
#         print(f"\n{wbl_type} ({len(wbl_subset)} rows):")

#         for col in complete_cols:
#             count = (wbl_subset[col] > 0).sum()
#             if count > 0:
#                 percentage = (count / len(wbl_subset)) * 100
#                 print(f"  {col}: {count} rows ({percentage:.1f}%)")

# # Analyze Pending rows if they exist
# pending_cols = ['Pending Student Interactions', 'Internships in Progress']
# pending_rows = dfmain[dfmain['Placement Status Category'] == 'Pending']

# if len(pending_rows) > 0:
#     print("\n" + "="*100)
#     print("PENDING ROWS - WBL Type to Numeric Column Distribution")
#     print("="*100)

#     for wbl_type in sorted(pending_rows['WBL Opportunity Type'].dropna().unique()):
#         wbl_subset = pending_rows[pending_rows['WBL Opportunity Type'] == wbl_type]

#         # Only show if there are non-zero values
#         has_values = (wbl_subset[pending_cols] > 0).any().any()

#         if has_values:
#             print(f"\n{wbl_type} ({len(wbl_subset)} rows):")

#             for col in pending_cols:
#                 count = (wbl_subset[col] > 0).sum()
#                 if count > 0:
#                     percentage = (count / len(wbl_subset)) * 100
#                     print(f"  {col}: {count} rows ({percentage:.1f}%)")

In [22]:
# Define the complete columns we're checking
complete_cols = ['Complete Student Trainings', 'Complete Student Interactions',
                 'Complete Student Internships']

# Track remapped rows
remapped_rows = []

# Process Completed rows
completed_rows = dfmain['Placement Status Category'] == 'Completed'

for idx in dfmain[completed_rows].index:
    row = dfmain.loc[idx].copy()
    wbl_type = row['WBL Opportunity Type']

    if wbl_type not in wbl_to_column_map_completed:
        continue

    correct_column = wbl_to_column_map_completed[wbl_type]

    # Check if any wrong columns have values
    for col in complete_cols:
        if col != correct_column and row[col] > 0:
            # EXCEPTION: Allow both Complete Staff Trainings and Complete Student Trainings
            if {col, correct_column} == {'Complete Staff Trainings', 'Complete Student Trainings'}:
                continue  # Skip this - it's a valid combination

            # Record the change
            remapped_rows.append({
                'index': idx,
                'WBL Opportunity Type': wbl_type,
                'Placement Status Category': 'Completed',
                'Wrong Column': col,
                'Correct Column': correct_column,
                'Value Moved': row[col],
                'Before': {c: row[c] for c in complete_cols},
            })

            # Move value from wrong column to correct column
            value = row[col]
            dfmain.loc[idx, correct_column] = dfmain.loc[idx, correct_column] + value
            dfmain.loc[idx, col] = 0

# Process Pending rows
pending_cols = ['Pending Student Interactions', 'Internships in Progress']
pending_rows = dfmain['Placement Status Category'] == 'Pending'

for idx in dfmain[pending_rows].index:
    row = dfmain.loc[idx].copy()
    wbl_type = row['WBL Opportunity Type']

    if wbl_type not in wbl_to_column_map_pending:
        continue

    correct_column = wbl_to_column_map_pending[wbl_type]

    # Check if any wrong columns have values
    for col in pending_cols:
        if col != correct_column and row[col] > 0:
            # Record the change
            remapped_rows.append({
                'index': idx,
                'WBL Opportunity Type': wbl_type,
                'Placement Status Category': 'Pending',
                'Wrong Column': col,
                'Correct Column': correct_column,
                'Value Moved': row[col],
                'Before': {c: row[c] for c in pending_cols},
            })

            # Move value from wrong column to correct column
            value = row[col]
            dfmain.loc[idx, correct_column] = dfmain.loc[idx, correct_column] + value
            dfmain.loc[idx, col] = 0

# # Create dataframe of remapped rows
# if len(remapped_rows) > 0:
#     remapped_df = pd.DataFrame(remapped_rows)
#     print(f"Remapped {len(remapped_rows)} rows to correct numeric columns")
#     print("\nRemapped rows:")
#     display(remapped_df)
# else:
#     print("No rows needed remapping - all values already in correct columns!")

### Fill All-Zero Rows with Respective Values

#### Main

In [23]:
# Find all-zero rows
all_zeros = (dfmain[numeric_cols] == 0).all(axis=1)

# Get WBL types that have all-zero rows
wbl_types_with_zeros = dfmain[all_zeros]['WBL Opportunity Type'].unique()

# Show value counts of WBL Opportunity Type for all-zero rows
print("\nWBL Opportunity Type breakdown for all-zero rows:")
print(dfmain[all_zeros]['WBL Opportunity Type'].value_counts())
print(f"\nTotal all-zero rows: {all_zeros.sum()}")


WBL Opportunity Type breakdown for all-zero rows:
WBL Opportunity Type
Speakers Bureau                  489
Professionalism 101 Training     141
Site Visit                       134
Class Presentation                98
e-WBL Class Presentation          91
Internship 60                     62
Regional Advisory Meeting         42
Student Training                   9
Job Shadow                         8
Event                              5
e-WBL Informational Interview      5
Site Visit - Staff                 4
Industry Sponsored Project         1
Career Story Video                 1
Apprenticeship                     1
Name: count, dtype: int64

Total all-zero rows: 1091


#### All 1's

In [24]:
# Create a copy to test this method
dfmain_test = dfmain.copy()

# Fill all-zero rows with 1's
for idx in dfmain_test[all_zeros].index:
    row = dfmain_test.loc[idx]
    wbl_type = row['WBL Opportunity Type']
    category = row['Placement Status Category']

    if category == 'Completed':
        if wbl_type in wbl_to_column_map_completed:
            dfmain_test.loc[idx, wbl_to_column_map_completed[wbl_type]] = 1
    elif category == 'Pending':
        if wbl_type in wbl_to_column_map_pending:
            dfmain_test.loc[idx, wbl_to_column_map_pending[wbl_type]] = 1
    elif category == 'Declined':
        dfmain_test.loc[idx, 'Declined Student Interactions'] = 1
    elif category == 'Cancelled':
        dfmain_test.loc[idx, 'Cancelled Student Interactions'] = 1

# Calculate metrics
total_completed = dfmain_test['Complete Student Interactions'].sum() + dfmain_test['Complete Student Trainings'].sum() + dfmain_test['Complete Student Internships'].sum()
total_declined = dfmain_test['Declined Student Interactions'].sum()
total_cancelled = dfmain_test['Cancelled Student Interactions'].sum()
grand_total = total_completed + total_declined + total_cancelled

# Store results
method_1_results = {
    'Method': 'All 1s',
    'Total Completed': total_completed,
    'Total Declined': total_declined,
    'Total Cancelled': total_cancelled,
    'Grand Total': grand_total
}

#### Min

In [25]:
# Create a copy to test this method
dfmain_test = dfmain.copy()

# Calculate minimum numeric sum by WBL Opportunity Type (excluding zeros)
wbl_minimums = dfmain_test[dfmain_test['_numeric_sum'] > 0].groupby('WBL Opportunity Type')['_numeric_sum'].min()

print("Minimum values by WBL Opportunity Type (for types with all-zero rows):")
for wbl_type in sorted(wbl_types_with_zeros):
    if pd.notna(wbl_type):
        min_val = wbl_minimums.get(wbl_type, 1)
        print(f"  {wbl_type}: {min_val}")

# Fill all-zero rows with minimum values
for idx in dfmain_test[all_zeros].index:
    row = dfmain_test.loc[idx]
    wbl_type = row['WBL Opportunity Type']
    category = row['Placement Status Category']

    min_value = wbl_minimums.get(wbl_type, 1)

    if category == 'Completed':
        if wbl_type in wbl_to_column_map_completed:
            dfmain_test.loc[idx, wbl_to_column_map_completed[wbl_type]] = min_value
    elif category == 'Pending':
        if wbl_type in wbl_to_column_map_pending:
            dfmain_test.loc[idx, wbl_to_column_map_pending[wbl_type]] = min_value
    elif category == 'Declined':
        dfmain_test.loc[idx, 'Declined Student Interactions'] = min_value
    elif category == 'Cancelled':
        dfmain_test.loc[idx, 'Cancelled Student Interactions'] = min_value

# Calculate metrics
total_completed = dfmain_test['Complete Student Interactions'].sum() + dfmain_test['Complete Student Trainings'].sum() + dfmain_test['Complete Student Internships'].sum()
total_declined = dfmain_test['Declined Student Interactions'].sum()
total_cancelled = dfmain_test['Cancelled Student Interactions'].sum()
grand_total = total_completed + total_declined + total_cancelled

# Store results
method_2_results = {
    'Method': 'Minimum',
    'Total Completed': total_completed,
    'Total Declined': total_declined,
    'Total Cancelled': total_cancelled,
    'Grand Total': grand_total
}

Minimum values by WBL Opportunity Type (for types with all-zero rows):
  Apprenticeship: 1.0
  Career Story Video: 1.0
  Class Presentation: 1.0
  Event: 1.0
  Industry Sponsored Project: 1.0
  Internship 60: 1.0
  Job Shadow: 1.0
  Professionalism 101 Training: 1.0
  Regional Advisory Meeting: 1
  Site Visit: 1.0
  Site Visit - Staff: 1
  Speakers Bureau: 10.0
  Student Training: 1.0
  e-WBL Class Presentation: 1.0
  e-WBL Informational Interview: 1.0


#### 25th Quartile

In [26]:
# Create a copy to test this method
dfmain_test = dfmain.copy()

# Calculate 25th percentile numeric sum by WBL Opportunity Type (excluding zeros)
wbl_25th_percentile = dfmain_test[dfmain_test['_numeric_sum'] > 0].groupby('WBL Opportunity Type')['_numeric_sum'].quantile(0.25)

print("25th Percentile values by WBL Opportunity Type (for types with all-zero rows):")
for wbl_type in sorted(wbl_types_with_zeros):
    if pd.notna(wbl_type):
        percentile_val = wbl_25th_percentile.get(wbl_type, 1)
        print(f"  {wbl_type}: {percentile_val}")

# Fill all-zero rows with 25th percentile values
for idx in dfmain_test[all_zeros].index:
    row = dfmain_test.loc[idx]
    wbl_type = row['WBL Opportunity Type']
    category = row['Placement Status Category']

    percentile_value = wbl_25th_percentile.get(wbl_type, 1)

    if category == 'Completed':
        if wbl_type in wbl_to_column_map_completed:
            dfmain_test.loc[idx, wbl_to_column_map_completed[wbl_type]] = percentile_value
    elif category == 'Pending':
        if wbl_type in wbl_to_column_map_pending:
            dfmain_test.loc[idx, wbl_to_column_map_pending[wbl_type]] = percentile_value
    elif category == 'Declined':
        dfmain_test.loc[idx, 'Declined Student Interactions'] = percentile_value
    elif category == 'Cancelled':
        dfmain_test.loc[idx, 'Cancelled Student Interactions'] = percentile_value

# Calculate metrics
total_completed = dfmain_test['Complete Student Interactions'].sum() + dfmain_test['Complete Student Trainings'].sum() + dfmain_test['Complete Student Internships'].sum()
total_declined = dfmain_test['Declined Student Interactions'].sum()
total_cancelled = dfmain_test['Cancelled Student Interactions'].sum()
grand_total = total_completed + total_declined + total_cancelled

# Store results
method_3_results = {
    'Method': '25th Percentile',
    'Total Completed': total_completed,
    'Total Declined': total_declined,
    'Total Cancelled': total_cancelled,
    'Grand Total': grand_total
}

25th Percentile values by WBL Opportunity Type (for types with all-zero rows):
  Apprenticeship: 1.0
  Career Story Video: 1.0
  Class Presentation: 15.0
  Event: 6.0
  Industry Sponsored Project: 1.0
  Internship 60: 1.0
  Job Shadow: 1.0
  Professionalism 101 Training: 1.0
  Regional Advisory Meeting: 1
  Site Visit: 10.0
  Site Visit - Staff: 1
  Speakers Bureau: 35.0
  Student Training: 3.0
  e-WBL Class Presentation: 4.0
  e-WBL Informational Interview: 1.0


#### Median

In [27]:
# Create a copy to test this method
dfmain_test = dfmain.copy()

# Calculate median numeric sum by WBL Opportunity Type (excluding zeros)
wbl_medians = dfmain_test[dfmain_test['_numeric_sum'] > 0].groupby('WBL Opportunity Type')['_numeric_sum'].median()

print("Median values by WBL Opportunity Type (for types with all-zero rows):")
for wbl_type in sorted(wbl_types_with_zeros):
    if pd.notna(wbl_type):
        median_val = wbl_medians.get(wbl_type, 1)
        print(f"  {wbl_type}: {median_val}")

# Fill all-zero rows with median values
for idx in dfmain_test[all_zeros].index:
    row = dfmain_test.loc[idx]
    wbl_type = row['WBL Opportunity Type']
    category = row['Placement Status Category']

    median_value = wbl_medians.get(wbl_type, 1)

    if category == 'Completed':
        if wbl_type in wbl_to_column_map_completed:
            dfmain_test.loc[idx, wbl_to_column_map_completed[wbl_type]] = median_value
    elif category == 'Pending':
        if wbl_type in wbl_to_column_map_pending:
            dfmain_test.loc[idx, wbl_to_column_map_pending[wbl_type]] = median_value
    elif category == 'Declined':
        dfmain_test.loc[idx, 'Declined Student Interactions'] = median_value
    elif category == 'Cancelled':
        dfmain_test.loc[idx, 'Cancelled Student Interactions'] = median_value

# Calculate metrics
total_completed = dfmain_test['Complete Student Interactions'].sum() + dfmain_test['Complete Student Trainings'].sum() + dfmain_test['Complete Student Internships'].sum()
total_declined = dfmain_test['Declined Student Interactions'].sum()
total_cancelled = dfmain_test['Cancelled Student Interactions'].sum()
grand_total = total_completed + total_declined + total_cancelled

# Store results
method_4_results = {
    'Method': 'Median',
    'Total Completed': total_completed,
    'Total Declined': total_declined,
    'Total Cancelled': total_cancelled,
    'Grand Total': grand_total
}

Median values by WBL Opportunity Type (for types with all-zero rows):
  Apprenticeship: 1.0
  Career Story Video: 1.0
  Class Presentation: 25.0
  Event: 15.0
  Industry Sponsored Project: 5.5
  Internship 60: 1.0
  Job Shadow: 1.0
  Professionalism 101 Training: 1.0
  Regional Advisory Meeting: 1
  Site Visit: 15.0
  Site Visit - Staff: 1
  Speakers Bureau: 51.0
  Student Training: 20.0
  e-WBL Class Presentation: 10.0
  e-WBL Informational Interview: 1.0


#### Compare all methods side-by-side

In [28]:
# Combine all results into a comparison dataframe
comparison_df = pd.DataFrame([
    method_1_results,
    method_2_results,
    method_3_results,
    method_4_results
])

# Set Method as index for cleaner display
comparison_df = comparison_df.set_index('Method')

print("="*100)
print("FILLING METHOD COMPARISON - STUDENT ENGAGEMENT METRICS")
print("="*100)
print("\n")
print(comparison_df.to_string())

# Show differences from minimum method (baseline)
print("\n" + "="*100)
print("DIFFERENCES FROM 'ALL 1s' METHOD (Baseline)")
print("="*100)

baseline = comparison_df.loc['All 1s']
differences = comparison_df - baseline

print("\n")
print(differences.to_string())

FILLING METHOD COMPARISON - STUDENT ENGAGEMENT METRICS


                 Total Completed  Total Declined  Total Cancelled  Grand Total
Method                                                                        
All 1s                   50836.0          4456.0           1737.0      57029.0
Minimum                  50935.0          8668.0           1764.0      61367.0
25th Percentile          51596.0         22782.0           1855.0      76233.0
Median                   52427.0         31932.0           1930.0      86289.0

DIFFERENCES FROM 'ALL 1s' METHOD (Baseline)


                 Total Completed  Total Declined  Total Cancelled  Grand Total
Method                                                                        
All 1s                       0.0             0.0              0.0          0.0
Minimum                     99.0          4212.0             27.0       4338.0
25th Percentile            760.0         18326.0            118.0      19204.0
Median                    1

In [29]:
# Current Method: Minimum + Median

# Calculate median numeric sum by WBL Opportunity Type (excluding zeros)
wbl_medians = dfmain[dfmain['_numeric_sum'] > 0].groupby('WBL Opportunity Type')['_numeric_sum'].median()

# Calculate minimum numeric sum by WBL Opportunity Type (excluding zeros)
wbl_minimums = dfmain[dfmain['_numeric_sum'] > 0].groupby('WBL Opportunity Type')['_numeric_sum'].min()

# Fill all-zero rows with median values
for idx in dfmain[all_zeros].index:
    row = dfmain.loc[idx]
    wbl_type = row['WBL Opportunity Type']
    category = row['Placement Status Category']

    median_value = wbl_medians.get(wbl_type, 1)

    min_value = wbl_minimums.get(wbl_type, 1)

    if category == 'Completed':
        if wbl_type in wbl_to_column_map_completed:
            dfmain.loc[idx, wbl_to_column_map_completed[wbl_type]] = median_value
    elif category == 'Pending':
        if wbl_type in wbl_to_column_map_pending:
            dfmain.loc[idx, wbl_to_column_map_pending[wbl_type]] = min_value
    elif category == 'Declined':
        dfmain.loc[idx, 'Declined Student Interactions'] = min_value
    elif category == 'Cancelled':
        dfmain.loc[idx, 'Cancelled Student Interactions'] = min_value

In [30]:
# Drop temporary numeric sum column
dfmain = dfmain.drop(columns=['_numeric_sum'])

In [31]:
# Verify
all_zeros_after = (dfmain[numeric_cols] == 0).all(axis=1)
print(f"\nAll-zero rows before filling: {all_zeros.sum()}")
print(f"All-zero rows after filling: {all_zeros_after.sum()}")


All-zero rows before filling: 1091
All-zero rows after filling: 0


# Duplicate Handling & Pending Events

In [32]:
# Define base matching fields for duplicate detection
base_match_fields = [
    'Business Champion Name',
    'Student Sponsor Name',
    'WBL Opportunity Type',
    'District',
    'Event Title',
    'School or Program Site'
]

# Create temporary column for numeric sum
dfmain['_numeric_sum'] = dfmain[numeric_cols].sum(axis=1)

# Find duplicates and decide which to keep
rows_to_drop = []

for idx in dfmain.index:
    row = dfmain.loc[idx]

    # Look for matches in earlier rows
    matches = dfmain[
        (dfmain.index < idx) &
        (dfmain['Business Champion Name'] == row['Business Champion Name']) &
        (dfmain['Student Sponsor Name'] == row['Student Sponsor Name']) &
        (dfmain['WBL Opportunity Type'] == row['WBL Opportunity Type']) &
        (dfmain['District'] == row['District']) &
        (dfmain['Event Title'] == row['Event Title']) &
        (dfmain['School or Program Site'] == row['School or Program Site']) &
        (dfmain['_numeric_sum'] == row['_numeric_sum']) &
        (
            (dfmain['Initiation Date'] == row['Initiation Date']) |
            (dfmain['Derived Event Date'] == row['Derived Event Date'])
        )
    ]

    if len(matches) > 0:
        orig_idx = matches.index[0]
        orig_status = dfmain.loc[orig_idx, 'Placement Status']
        dup_status = row['Placement Status']

        # Rule 1: Prefer 'Completed' status
        if orig_status == 'Completed' and dup_status != 'Completed':
            rows_to_drop.append(idx)
        elif dup_status == 'Completed' and orig_status != 'Completed':
            rows_to_drop.append(orig_idx)
        else:
            # Rule 2: Choose the one with least nulls
            orig_null_count = dfmain.loc[orig_idx].isna().sum()
            dup_null_count = row.isna().sum()

            if orig_null_count < dup_null_count:
                rows_to_drop.append(idx)
            elif dup_null_count < orig_null_count:
                rows_to_drop.append(orig_idx)
            else:
                # Rule 3: Keep the most recent based on Derived Event Date
                orig_date = dfmain.loc[orig_idx, 'Derived Event Date']
                dup_date = row['Derived Event Date']

                if pd.isna(orig_date) and pd.isna(dup_date):
                    rows_to_drop.append(idx)
                elif pd.isna(orig_date):
                    rows_to_drop.append(orig_idx)
                elif pd.isna(dup_date):
                    rows_to_drop.append(idx)
                elif dup_date > orig_date:
                    rows_to_drop.append(orig_idx)
                else:
                    rows_to_drop.append(idx)

# Remove duplicates from rows_to_drop list
rows_to_drop = list(set(rows_to_drop))

# Drop duplicates
dfmain = dfmain.drop(rows_to_drop)

# Drop temporary numeric sum column
dfmain = dfmain.drop(columns=['_numeric_sum'])

In [33]:
# Fix 2019-2020 Internship rows marked as Declined that should be Completed
declined_should_be_completed = dfmain[
    (dfmain['Placement Status Category'] == 'Declined') &
    (dfmain['School Year'] == '2019-2020') &
    (dfmain['WBL Opportunity Type'].isin(['Internship 60', 'Internship 120', 'Internship 320'])) &
    (dfmain['Complete Student Interactions'] > 0)
].index

dfmain.loc[declined_should_be_completed, 'Placement Status'] = 'Completed'
dfmain.loc[declined_should_be_completed, 'Placement Status Category'] = 'Completed'
dfmain.loc[declined_should_be_completed, 'Complete Student Internships'] = dfmain.loc[declined_should_be_completed, 'Complete Student Interactions']
dfmain.loc[declined_should_be_completed, 'Complete Student Interactions'] = 0

In [34]:
# Mark 'Internship In Process' 2024-2025 rows as 'Completed'
internship_2024_25 = (
    (dfmain['Placement Status'] == 'Internship In Process') &
    (dfmain['School Year'] == '2024-2025')
)
dfmain.loc[internship_2024_25, 'Placement Status'] = 'Completed'

# Transfer internship counts to Complete Student Internships for these rows
dfmain.loc[internship_2024_25, 'Complete Student Internships'] = dfmain.loc[internship_2024_25, 'Internships in Progress']
dfmain.loc[internship_2024_25, 'Internships in Progress'] = 0

# Reclassify remaining pending statuses as "Declined - Unfinished"
pending_statuses = [
    'Initial Contact Made', 'Pending-Scheduling', 'Scheduled Interview',
    'Internship In Process', 'Scheduled Event (Pending Completion)'
]
dfmain.loc[dfmain['Placement Status'].isin(pending_statuses), 'Placement Status'] = 'Declined - Unfinished'

# Transfer pending counts to declined counts for reclassified rows
dfmain.loc[
    dfmain['Placement Status'] == 'Declined - Unfinished',
    'Declined Student Interactions'
] = (
    dfmain.loc[dfmain['Placement Status'] == 'Declined - Unfinished', 'Pending Student Interactions'] +
    dfmain.loc[dfmain['Placement Status'] == 'Declined - Unfinished', 'Internships in Progress']
)

# Zero out the pending columns for these rows
dfmain.loc[dfmain['Placement Status'] == 'Declined - Unfinished', 'Pending Student Interactions'] = 0
dfmain.loc[dfmain['Placement Status'] == 'Declined - Unfinished', 'Internships in Progress'] = 0

# Drop the now-irrelevant pending columns
dfmain = dfmain.drop(columns=['Pending Student Interactions', 'Internships in Progress'])

# Match Placement Status Category with new Placement Status: 'Declined - Unfinished'
dfmain['Placement Status Category'] = dfmain['Placement Status'].apply(categorize_placement_status)

# Verify
print(f"Rows dropped as duplicates: {len(rows_to_drop)}")
print(f"Final row count: {len(dfmain)}")
print(f"\nPlacement Status counts:\n")
print(dfmain['Placement Status Category'].value_counts(), "\n")
print(dfmain['Placement Status'].value_counts())

Rows dropped as duplicates: 205
Final row count: 9922

Placement Status counts:

Placement Status Category
Completed    7651
Declined     2190
Cancelled      81
Name: count, dtype: int64 

Placement Status
Completed                           7651
Declined-Applicant                   475
Declined - Business Unresponsive     225
Declined/Cancelled-Other             224
Declined-Business Scheduling         220
Declined-Business                    210
Declined- Student Applicant          181
Declined - Student Profile           176
Declined - Student Other             113
Declined - Student Unresponsive       90
Declined - Staff Scheduling           76
Declined-Intern NOT Selected          66
Cancelled-COVID                       63
Declined - Staff Applicant            35
Declined - Unfinished                 35
Declined - Staff Unresponsive         26
Declined-Opportunity FULL             24
Cancelled-Weather                     17
Terminated                            14
Cancelled-Illne

# Standardizing District Names and Reorganize Column Structure

In [35]:
# Standardize District Names

district_mapping = {
    'D11': 'Colorado Springs (D11)',
    'D20': 'Academy (D20)',
    'D49': 'El Paso County (D49)',
    'D2': 'Harrison (D2)',
    'D3': 'Widefield (D3)',
    'D8': 'Fountain-Fort Carson (D8)',
    'D12': 'Cheyenne Mountain (D12)',
    'D14': 'Manitou Springs (D14)',
    'D38': 'Lewis-Palmer (D38)',
    'CEC-CS': 'Colorado Springs Early College (CEC-CS)',
    'BLR': 'Banning Lewis Ranch (BLR)',
    'WPSD': 'Woodland Park (WPSD)',
    'Ellicott': 'Ellicott (D22)',
    'Peyton': 'Peyton (D23JT)',
    'Calhan': 'Calhan (RJ-1)',
    'Atlas Prep': 'Atlas Preparatory',
    'CPA/PPOS': 'CO Digital BOCES (CPA/PPOS)',
    'PTEC': 'Power Technical (PTEC)',
    'Goal H.S.': 'Goal High School',
    'Mon Impact': 'Monumental Impact',
    'Peak Ed': 'Peak Education',
    'Miami-Yoder': 'Miami-Yoder (JT-60)',
    'ECA': 'Evangel Christian Academy',
    'MET': 'Mountain Employment Training',
    'TCA': 'The Classical Academy',
    'CCV': 'Cripple Creek-Victor (RE-1)',
    'DYS': 'Division of Youth Services',
    'Vanguard': 'Vanguard School',
    'Homeschool': 'Homeschool',
    'Various': 'Various',
    'BBBS': 'Big Brothers Big Sisters',
    'Roundup': 'Roundup School'
}

dfmain['District'] = dfmain['District'].replace(district_mapping)

# Reorganize Column Structure

column_order = [
    'School Year',
    'Derived Event Date',
    'Placement Status Category',
    'Placement Status',
    'WBL Opportunity Type',
    'Event Title',
    'Business Champion Name',
    'Student Sponsor Name',
    'District',
    'School or Program Site',
    'Complete Student Interactions',
    'Complete Student Trainings',
    'Complete Student Internships',
    'Declined Student Interactions',
    'Cancelled Student Interactions',
    'Complete Staff Trainings',
    'Initiation Date',
    'Status Update Date',
    'Event Date or Start Date',
]

dfmain = dfmain[column_order]

# Final Data Summary

In [36]:
print("=" * 60)
print("FINAL DATASET SUMMARY")
print("=" * 60)

# Size
print(f"\nSize: {dfmain.shape[0]:,} rows × {dfmain.shape[1]} columns")

# Date Range
print(f"\nDate Range: {dfmain['Derived Event Date'].min().strftime('%B %d, %Y')} - {dfmain['Derived Event Date'].max().strftime('%B %d, %Y')}")

# Records by Year
print("\nRecords by Year:")
year_counts = dfmain['School Year'].value_counts().sort_index()
for year, count in year_counts.items():
    print(f"  {year}: {count:,}")

# Status Distribution (by Category)
print("\nStatus Distribution (by Category):")
status_counts = dfmain['Placement Status Category'].value_counts()
for status, count in status_counts.items():
    pct = count / len(dfmain) * 100
    print(f"  {status}: {count:,} ({pct:.1f}%)")

# Status Distribution (detailed)
print("\nStatus Distribution (detailed):")
detailed_status = dfmain['Placement Status'].value_counts()
for status, count in detailed_status.items():
    pct = count / len(dfmain) * 100
    print(f"  {status}: {count:,} ({pct:.1f}%)")

# Student Engagement Metrics
print("\nStudent Engagement Metrics:")
total_completed = dfmain['Complete Student Interactions'].sum() + dfmain['Complete Student Trainings'].sum() + dfmain['Complete Student Internships'].sum()
total_declined = dfmain['Declined Student Interactions'].sum()
total_cancelled = dfmain['Cancelled Student Interactions'].sum()
grand_total = total_completed + total_declined + total_cancelled

print(f"  Total Completed: {total_completed:,.0f}")
print(f"  Total Declined: {total_declined:,.0f}")
print(f"  Total Cancelled: {total_cancelled:,.0f}")
print(f"  Grand Total: {grand_total:,.0f} student engagements")

# Average students per completed event
completed_events = dfmain[dfmain['Placement Status Category'] == 'Completed']
avg_students = total_completed / len(completed_events) if len(completed_events) > 0 else 0
print(f"  Average Students per Completed Event: {avg_students:.1f}")

# Staff Metrics
print("\nStaff Metrics:")
print(f"  Staff Trainings: {dfmain['Complete Staff Trainings'].sum():,.0f}")

# Top 5 Districts
print("\nTop 5 Districts:")
top_districts = dfmain['District'].value_counts().head(5)
for i, (district, count) in enumerate(top_districts.items(), 1):
    print(f"  {i}. {district}: {count:,} events")

# Top 5 WBL Types
print("\nTop 5 WBL Opportunity Types:")
top_wbl = dfmain['WBL Opportunity Type'].value_counts().head(5)
for i, (wbl, count) in enumerate(top_wbl.items(), 1):
    print(f"  {i}. {wbl}: {count:,} events")

# Top 5 Businesses
print("\nTop 5 Business Partners:")
top_business = dfmain['Business Champion Name'].value_counts().head(5)
for i, (business, count) in enumerate(top_business.items(), 1):
    print(f"  {i}. {business}: {count:,} events")

print("\n" + "=" * 60)

FINAL DATASET SUMMARY

Size: 9,922 rows × 19 columns

Date Range: January 07, 2019 - September 22, 2025

Records by Year:
  2019-2020: 330
  2020-2021: 1,905
  2021-2022: 1,997
  2022-2023: 2,743
  2023-2024: 1,502
  2024-2025: 1,445

Status Distribution (by Category):
  Completed: 7,651 (77.1%)
  Declined: 2,190 (22.1%)
  Cancelled: 81 (0.8%)

Status Distribution (detailed):
  Completed: 7,651 (77.1%)
  Declined-Applicant: 475 (4.8%)
  Declined - Business Unresponsive: 225 (2.3%)
  Declined/Cancelled-Other: 224 (2.3%)
  Declined-Business Scheduling: 220 (2.2%)
  Declined-Business: 210 (2.1%)
  Declined- Student Applicant: 181 (1.8%)
  Declined - Student Profile: 176 (1.8%)
  Declined - Student Other: 113 (1.1%)
  Declined - Student Unresponsive: 90 (0.9%)
  Declined - Staff Scheduling: 76 (0.8%)
  Declined-Intern NOT Selected: 66 (0.7%)
  Cancelled-COVID: 63 (0.6%)
  Declined - Staff Applicant: 35 (0.4%)
  Declined - Unfinished: 35 (0.4%)
  Declined - Staff Unresponsive: 26 (0.3%)
  D

# Export

In [37]:
import os

output_path = '/content/drive/MyDrive/Work/BEA/2025 BEA Data Project Shared Folder/Data/(Main) Data Sources/Existing/PPBEA Pipeline/Cleaned/'
os.makedirs(output_path, exist_ok=True)

output_file = f'{output_path}PPBEA_Pipeline_2019-2025_Cleaned.csv'
dfmain.to_csv(output_file, index=False)

print(f"Cleaned data saved to: {output_file}")
print(f"Final shape: {dfmain.shape}")

Cleaned data saved to: /content/drive/MyDrive/Work/BEA/2025 BEA Data Project Shared Folder/Data/(Main) Data Sources/Existing/PPBEA Pipeline/Cleaned/PPBEA_Pipeline_2019-2025_Cleaned.csv
Final shape: (9922, 19)
